Try differential testing with a random intercept model for sample and maybe chip!

In [1]:
import scanpy.api as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from copy import deepcopy
import seaborn as sb
from matplotlib import colors
from gprofiler import gprofiler

from rpy2.rinterface import RRuntimeWarning
from rpy2.robjects import pandas2ri
from IPython.core.interactiveshell import InteractiveShell


%load_ext rpy2.ipython

/app/python37/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
#sc.pl.reset_rcParams() #reset figure parameters
InteractiveShell.ast_node_interactivity = "all"
sc.settings.verbosity = 3
sc.logging.print_versions()

scanpy==1.5.1 anndata==0.7.5 umap==0.3.10 numpy==1.20.0 scipy==1.6.0 pandas==1.2.1 scikit-learn==0.24.1 statsmodels==0.12.2 python-igraph==0.8.3 louvain==0.6.1 leidenalg==0.7.0


In [3]:
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth", 800)

# Load the data

In [4]:
#User inputs
adata_file = './data/Ketamine_Single_Cell/Ketamine_full_scran_nb_full_analysis_final.h5ad'

In [5]:
adata = sc.read(adata_file)

In [6]:
adata.X.shape

(5013, 20670)

In [7]:
#Use raw data (here scran-log-normalized data)
adata.X = adata.raw.X

In [8]:
adata

AnnData object with n_obs × n_vars = 5013 × 20670
    obs: 'sample', 'chip', 'condition', 'chip_pair', 'candidate', 'microscopy_state', 'quality_signal', 'damage_signal', 'count_depth', 'n_genes', 'mt_frac', 'n_counts', 'size_factors', 'louvain_r1', 'louvain_final'
    var: 'gene_symbols', 'n_cells', 'dropout', 'mean_expr', 'log10_mean_expr', 'gini_index', 'ambient_genes', 'highly_variable_genes', 'expression_mean', 'dispersion', 'pseudogene_mask', 'mean_gene_exp'
    uns: 'ambient_genes_colors', 'chip_colors', 'chip_pair_colors', 'louvain', 'louvain_r1_colors', 'neighbors', 'pca', 'rank_genes_r1', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# Differential testing prep

First define the test function

In [9]:
%%R
require(edgeR)
require(limma)
require(Matrix)

run_limma = function(data_mat, obs) {
    cond <- as.factor(obs$condition)
    batch <- as.factor(obs$chip)

    dge <- edgeR::DGEList(data_mat, group=cond, sample=batch)
    dmat <- model.matrix(~cond + batch)

    #Check the rank of the data matrix:
    dmat_uniq <- unique(dmat)
    dmat_rank <- rankMatrix(dmat_uniq)[1]
    dmat_min_dim <- min(dim(dmat_uniq))
    rank_def <- dmat_min_dim-dmat_rank
    print(paste("Rank deficiency is:", rank_def))

    y <- new("EList")
    y$E <- dge

    print('run limma lmFit')
    fit <- limma::lmFit(y, design = dmat)

    print('run limma eBayes')
    fit <-  limma::eBayes(fit, trend = TRUE, robust = TRUE)

    tt <- limma::topTable(fit, coef='condTRUE', number=Inf, adjust.method='BH')

    return(tt)
}


R[write to console]: Loading required package: edgeR

R[write to console]: Loading required package: limma

R[write to console]: Loading required package: Matrix



# Loop over the clusters

In [10]:
# Prepare the data
clusters = adata.obs['louvain_final'].cat.categories
limma_results_signif = dict()
limma_results_full = dict()

In [11]:
adata.obs['louvain_final'].value_counts()

Astrocytes                       1746
Glut Neurons                     1024
Oligodendrocytes                  622
Unknown                           489
Endothelial                       303
Microglia                         275
OPCs                              233
Pericytes                          85
Vascular smooth muscle cells       46
Vascular leptomeningeal cells      40
Unknown 2                          36
GABA Neurons                       34
Blood                              32
Ependymal                          28
Perivascular macrophages           20
Name: louvain_final, dtype: int64

In [12]:
adata.obs.condition

C92579_CELL_0_10_8Yellow_AACCTTCGACT     True
C92579_CELL_0_14_8Blue_AACCAATCTCT      False
C92579_CELL_0_16_8Blue_AACCAACTAGA      False
C92579_CELL_0_17_8Blue_AACCAAGATTC      False
C92579_CELL_0_19_8Blue_AACCAATATAG      False
                                        ...  
C93411_CELL_9_61_8Yellow_AACTGACTATG     True
C93411_CELL_9_66_8Yellow_AACTCTTGGAG     True
C93411_CELL_9_70_8Yellow_AACTCTCAAGC     True
C93411_CELL_9_71_8Yellow_AACTCTGCGTA     True
C93411_CELL_9_8_8Blue_AAGAGAGCTCT       False
Name: condition, Length: 5013, dtype: bool

In [13]:
for clust in clusters:
    print("Running limma for cluster `{}`".format(clust))

    # Subset the dataset
    adata_tmp = adata[adata.obs['louvain_final'] == clust].copy()

    # Filter genes that are not expressed in at least 10% of cells
    sc.pp.filter_genes(adata_tmp, min_cells=np.round(adata_tmp.n_obs*0.1))
    print("Number of genes post filtering: {}".format(adata_tmp.n_vars))
    
    adata_tmp.var['mean_exp_ctrl'] = adata_tmp[adata_tmp.obs.condition==False].X.mean(axis=0)
    adata_tmp.var['mean_exp_test'] = adata_tmp[adata_tmp.obs.condition==True].X.mean(axis=0)
    
    # Generate the data object
    data_mat = pd.DataFrame(adata_tmp.X.T, index=adata_tmp.var_names, columns=adata_tmp.obs_names)
    obs = adata_tmp.obs

    # Run limma
    %R -i data_mat -i obs -o res res=run_limma(data_mat, obs)

    # Add gene symbols
    res['gene_symbol'] = adata_tmp.var['gene_symbols'][res.index]
    res['mean_exp_ctrl'] = adata_tmp.var.mean_exp_ctrl[res.index]
    res['mean_exp_test'] = adata_tmp.var.mean_exp_test[res.index] 
    # Store all results
    limma_results_full[clust] = res

    # Select only significant genes
    res = res[res['adj.P.Val'] < 0.05]

    # Store only data where ther are DEGs
    if res.shape[0] != 0:
        limma_results_signif[clust] = res

Running limma for cluster `Glut Neurons`


/app/python37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


filtered out 10900 genes that are detected in less than 102.0 cells
Number of genes post filtering: 9770


/app/python37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/app/python37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


[1] "Rank deficiency is: 0"
[1] "run limma lmFit"
[1] "run limma eBayes"


...

,logFC,AveExpr,t,P.Value,adj.P.Val,B
ENSMUSG00000029098.16,0.143678,0.097444,4.931538,0.000001,0.008834,4.920095
ENSMUSG00000033767.14,0.247299,0.258763,4.619418,0.000005,0.019561,3.643062
ENSMUSG00000019984.16,0.192955,0.217803,4.412355,0.000012,0.033436,2.838079
ENSMUSG00000059149.17,0.269862,0.391538,4.321824,0.000018,0.037416,2.496825
ENSMUSG00000081948.4,0.284425,0.499336,4.042643,0.000061,0.098350,1.485886
...,...,...,...,...,...,...
ENSMUSG00000028759.13,0.000089,0.844527,0.000864,0.999311,0.999743,-5.723319
ENSMUSG00000016831.11,-0.000037,0.174237,-0.000785,0.999374,0.999743,-5.723319
ENSMUSG00000006932.17,-0.000030,0.153960,-0.000625,0.999501,0.999747,-5.723319
ENSMUSG00000024943.8,-0.000006,0.065689,-0.000245,0.999805,0.999928,-5.723319


Running limma for cluster `Unknown 2`
filtered out 11596 genes that are detected in less than 4.0 cells
Number of genes post filtering: 9074


/app/python37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/app/python37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/app/python37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


[1] "Rank deficiency is: 0"
[1] "run limma lmFit"
[1] "run limma eBayes"


,logFC,AveExpr,t,P.Value,adj.P.Val,B
ENSMUSG00000036751.7,-1.123792,1.295439,-3.813197,0.000265,0.998639,-2.873801
ENSMUSG00000021057.15,-0.626025,3.935649,-3.469449,0.000836,0.998639,-3.164088
ENSMUSG00000027835.11,-0.751881,0.439436,-3.367980,0.001158,0.998639,-3.246932
ENSMUSG00000036315.13,0.446725,0.159691,3.279055,0.001533,0.998639,-3.318360
ENSMUSG00000022110.13,0.674481,0.330807,3.294399,0.001512,0.998639,-3.318667
...,...,...,...,...,...,...
ENSMUSG00000025817.12,0.000137,2.707576,0.000634,0.999495,0.999839,-4.771263
ENSMUSG00000043467.19,-0.000143,1.196880,-0.000564,0.999552,0.999839,-4.771263
ENSMUSG00000005501.14,0.000070,0.227277,0.000433,0.999656,0.999839,-4.771263
ENSMUSG00000037577.12,0.000072,0.505885,0.000340,0.999729,0.999839,-4.771263


Note that `.raw` data was used here. It was stored in the `.X` matrix above.

# Output results

Output the `louvain_final` results

In [14]:
clusters

Index(['Glut Neurons', 'GABA Neurons', 'Astrocytes', 'Oligodendrocytes',
       'OPCs', 'Microglia', 'Perivascular macrophages', 'Endothelial',
       'Ependymal', 'Pericytes', 'Vascular leptomeningeal cells',
       'Vascular smooth muscle cells', 'Blood', 'Unknown', 'Unknown 2'],
      dtype='object')

In [15]:
folder = './data/Ketamine_Single_Cell/Diff_testing/limma_final/'

In [16]:
for clust in clusters:
    filename = 'full_de_res_'+clust.replace('(','').replace(')','').replace(' ','_')+'.csv'
    limma_results_full[clust].to_csv(folder+filename)

    if clust in limma_results_signif:
        filename = 'significant_de_genes_'+clust.replace('(','').replace(')','').replace(' ','_')+'.csv'
        limma_results_signif[clust].to_csv(folder+filename)